# DeepRouteSet Testing


In [1]:
from os import environ
environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/home/valsp/miniconda3/envs/tf/lib"


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import Counter
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.backend import argmax
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

from DeepRouteSet_helper import *


2023-04-24 11:02:26.677247: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
cwd = Path().cwd()
n_values = 278
n_a = 64

In [4]:
with open(cwd.parent / "raw_data" / "holdStr_to_holdIx", 'rb') as f:
    holdStr_to_holdIx = pickle.load(f)
with open(cwd.parent / "raw_data" / "holdIx_to_holdStr", 'rb') as f:
    holdIx_to_holdStr = pickle.load(f)

## 3 - Generating moonboard problem

We already have trained model which has learned the patterns of climbing move and route set spirit. Lets now use this model to synthesize new route.
Importantly we reuse the `LSTM_cell` and `densor` layers from the model we trained.


### 3.1 - Loading the model

### 3.2 - Initialization of inference model
The following code creates the zero-valued vectors you will use to initialize `x` and the LSTM state variables `a` and `c`. 

In [5]:
x_initializer = np.random.rand(1, 1, n_values) / 100
a_initializer = np.random.rand(1, n_a) * 150
c_initializer = np.random.rand(1, n_a) /2

## 4.  Prediction with pretrained model

### 4.1 - DeepRouteSet generated one problem

In [6]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Generates a new climb using the inference model and a set of intial hidden and cell states.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, n_values), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, n_values), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices =  np.argmax(pred, axis = 2)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (Ty, n_values)
    results =  to_categorical(indices, num_classes = np.shape(x_initializer)[2])
    
    return results, indices

In [7]:
model_dir = cwd / 'DeepRouteSet'
inference_model = load_model(model_dir)
# inference_model.load_weights(cwd / 'DeepRouteSet_medium.h5')
# Check the inference model
# inference_model.summary()

In [14]:
#Initial conditions like x,a,c can be changed
x_initializer = np.random.rand(1, 1, n_values) / 100
a_initializer = np.random.rand(1, n_a) * 150
c_initializer = np.random.rand(1, n_a) /2

results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[2]) =", np.argmax(results[2]))
print("np.argmax(results[7]) =", np.argmax(results[7]))
print("list(indices[2:8]) =", list(indices[2:8]))

1/1 [==============================] - 0s 73ms/step
np.argmax(results[2]) = 0
np.argmax(results[7]) = 0
list(indices[2:8]) = [array([0]), array([0]), array([0]), array([0]), array([0]), array([0])]


In [15]:
print("DeepRouteSet's generated problem:")
for i in range(12):
    print(holdIx_to_holdStr[int(indices[i])])

DeepRouteSet's generated problem:
End
End
End
End
End
End
End
End
End
End
End
End


### 4.2 - Sanity Check 
Check and filter out some of the not right problem

In [12]:
sanityCheckAndOutput(indices, holdIx_to_holdStr, printError=True)

Repeat hand error ['A18-RH', 'E13-LH']


(False, ['A18-RH', 'E13-LH'], [17, 145])

### 4.3 - Plot 40 generated moonboard problems
Plot 40 generated moonboard problems 

Things to try:
1. change x, a, c initializer and see how it behave
2. Change the how many more benchmark should add into training set. Now it is 6 duplicate of benchMark problem

In [46]:
NUM_GEN = 40
passCount = 0
passGeneratedHandSequenceList = []
for i in range(NUM_GEN):
    x_initializer = np.zeros((1, 1, n_values))
    x_initializer = np.random.rand(1, 1, n_values) / 100
    a_initializer = np.random.rand(1, n_a) * 150
    c_initializer = np.random.rand(1, n_a) /2
    
    results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
    passCheck, outputListInString, outputListInIx = sanityCheckAndOutput(
        indices, holdIx_to_holdStr, printError = True)
    if passCheck:
        print(outputListInString)
        # plotAProblem(outputListInString)
        passCount = passCount + 1
        passGeneratedHandSequenceList.append(outputListInString)
print (f"{passCount} valid out of {NUM_GEN}")        

1/1 [==============================] - 0s 38ms/step
Repeat hand error ['F5-RH', 'F5-LH', 'F5-RH']
1/1 [==============================] - 0s 37ms/step
['A5-LH', 'A5-RH', 'C5-LH', 'E8-RH', 'E9-LH', 'H12-RH', 'G15-LH', 'K18-RH']
1/1 [==============================] - 0s 38ms/step
['F5-LH', 'F5-RH', 'B9-LH', 'E12-RH', 'C13-LH', 'D18-RH']
1/1 [==============================] - 0s 34ms/step
['E6-LH', 'E6-RH', 'F10-RH', 'C13-LH', 'D18-RH']
1/1 [==============================] - 0s 32ms/step
Repeat hand error ['K5-RH', 'H5-LH', 'H5-RH', 'B4-RH']
1/1 [==============================] - 0s 32ms/step
['F5-LH', 'F5-RH', 'B8-LH', 'F10-RH', 'C13-LH', 'E18-RH']
1/1 [==============================] - 0s 32ms/step
['F5-LH', 'F5-RH', 'H11-RH', 'C13-LH', 'E18-RH']
1/1 [==============================] - 0s 32ms/step
['E6-LH', 'E6-RH', 'F10-RH', 'C13-LH', 'D18-RH']
1/1 [==============================] - 0s 32ms/step
Repeat hand error ['B6-LH', 'B6-RH']
1/1 [==============================] - 0s 31ms/step
['J

In [47]:
len(passGeneratedHandSequenceList)

15

In [48]:
print(passGeneratedHandSequenceList[0])

['A5-LH', 'A5-RH', 'C5-LH', 'E8-RH', 'E9-LH', 'H12-RH', 'G15-LH', 'K18-RH']


### 4.4 - Produce move generator
To feed into GradeNet, we should transform from hand sequence to move sequence

In [49]:
# Feed in the hold feature.csv files
left_hold_feature_path = cwd.parent / 'raw_data' / 'hold_features_2016_LH.csv'
right_hold_feature_path = cwd.parent / 'raw_data' / 'hold_features_2016_RH.csv'

# convert the csv to saved pickle dictionary
LeftHandfeatures = pd.read_csv(left_hold_feature_path, dtype=str)
RightHandfeatures = pd.read_csv(right_hold_feature_path, dtype=str)
# convert features from pd dataframe to dictionary of left and right hand
RightHandfeature_dict = {}
LeftHandfeature_dict = {}
for index in RightHandfeatures.index:
    LeftHandfeature_item = LeftHandfeatures.loc[index]
    LeftHandfeature_dict[(int(LeftHandfeature_item['X_coord']), int(LeftHandfeature_item['Y_coord']))] = np.array(
        list(LeftHandfeature_item['Difficulties'])).astype(int)
    RightHandfeature_item = RightHandfeatures.loc[index]
    RightHandfeature_dict[(int(RightHandfeature_item['X_coord']), int(RightHandfeature_item['Y_coord']))] = np.array(
        list(RightHandfeature_item['Difficulties'])).astype(int)

In [51]:
# Output with a list of dictionary means moves
moveGeneratorFromStrList(passGeneratedHandSequenceList[0], string_mode = False)

[{'TargetHoldString': (2, 4),
  'TargetHoldHand': 0,
  'TargetHoldScore': array([5]),
  'RemainingHoldString': (0, 4),
  'RemainingHoldHand': 1,
  'RemainingHoldScore': array([4]),
  'dxdyRtoT': (2, 0),
  'MovingHoldString': (0, 4),
  'MovingHoldHand': 0,
  'MovingHoldScore': array([4]),
  'dxdyMtoT': (2, 0),
  'FootPlacement': [0, 0, 0, 1, 1, 1, 0],
  'MoveSuccessRate': array([5.44768087])},
 {'TargetHoldString': (4, 7),
  'TargetHoldHand': 1,
  'TargetHoldScore': array([8]),
  'RemainingHoldString': (2, 4),
  'RemainingHoldHand': 0,
  'RemainingHoldScore': array([5]),
  'dxdyRtoT': (2, 3),
  'MovingHoldString': (0, 4),
  'MovingHoldHand': 1,
  'MovingHoldScore': array([4]),
  'dxdyMtoT': (4, 3),
  'FootPlacement': [0, 0, 0, 1, 1, 1, 0],
  'MoveSuccessRate': array([14.84316105])},
 {'TargetHoldString': (4, 8),
  'TargetHoldHand': 0,
  'TargetHoldScore': array([3]),
  'RemainingHoldString': (4, 7),
  'RemainingHoldHand': 1,
  'RemainingHoldScore': array([8]),
  'dxdyRtoT': (0, 1),
  'M

### 4.5 - Generate moves and save the result (Don't overwrite)
Two things to save:
1. Move sequence -- For GradeNet and StyleNet purpose
2. List of holds -- For ploting purpose

In [55]:
save_path = cwd / 'DeepRouteSet_medium_out'

In [59]:
dim22Vec, listOfSavedSequence = moveGeneratorForAllGeneratedProblem(
    passGeneratedHandSequenceList,
    save_path,
    "MediumDeepRouteSet_v1_id",
    print_result=True,
)


completed MediumDeepRouteSet_v1_id1
completed MediumDeepRouteSet_v1_id2
completed MediumDeepRouteSet_v1_id3
completed MediumDeepRouteSet_v1_id4
completed MediumDeepRouteSet_v1_id5
skip duplicate ['E6-LH', 'E6-RH', 'F10-RH', 'C13-LH', 'D18-RH']
completed MediumDeepRouteSet_v1_id6
completed MediumDeepRouteSet_v1_id7
completed MediumDeepRouteSet_v1_id8
completed MediumDeepRouteSet_v1_id9
completed MediumDeepRouteSet_v1_id10
completed MediumDeepRouteSet_v1_id11
completed MediumDeepRouteSet_v1_id12
completed MediumDeepRouteSet_v1_id13
skip duplicate ['E6-LH', 'E6-RH', 'F10-RH', 'C13-LH', 'D18-RH']
result saved. Store  13 out of 15


In [ ]:
# TODO: investigate why only some are

In [ ]:
print(dim22Vec[next(iter(dim22Vec))])

[[ 5.          2.          3.          0.        ]
 [ 9.         12.         14.         17.        ]
 [ 1.          0.          1.          0.        ]
 [ 5.          8.          4.          5.        ]
 [ 4.          5.          2.          3.        ]
 [ 5.          9.         12.         14.        ]
 [ 5.          5.          8.          4.        ]
 [ 1.         -3.          1.         -3.        ]
 [ 4.          3.          2.          3.        ]
 [ 4.          4.          5.          2.        ]
 [ 5.          5.          9.         12.        ]
 [ 3.          5.          5.          8.        ]
 [ 1.         -2.         -2.         -2.        ]
 [ 4.          7.          5.          5.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.        ]
 [ 0.          0.          1.          0.        ]
 [ 1.          0.          0.          0.        ]
 [ 1.          1.          0.          0.        ]
 [ 1.          0.          0.  

In [ ]:
dim22Vec[next(iter(dim22Vec))].shape

(22, 4)

In [ ]:
save_pickle(listOfSavedSequence, cwd / 'DeepRouteSet_medium_out_seq')

### 4.6 - Plot stored file

In [52]:
with open(cwd.parent / "out" / "DeepRouteSet_medium_out_seq", 'rb') as f:
    listOfSavedSequence = pickle.load(f)

In [53]:
listOfSavedSequence[0]

['E6-LH', 'E6-RH', 'I10-RH', 'C13-LH', 'E16-RH', 'A18-LH']

In [54]:
count = 1
for seq in listOfSavedSequence:
    print ("MediumRouteSet_v1_id", count)
    print ("beta", seq)
    plotAProblem(seq)
    count = count + 1

MediumRouteSet_v1_id 1
beta ['E6-LH', 'E6-RH', 'I10-RH', 'C13-LH', 'E16-RH', 'A18-LH']
MediumRouteSet_v1_id 2
beta ['H5-LH', 'H5-RH', 'E6-LH', 'H8-RH', 'F10-LH', 'H12-RH', 'G15-LH', 'K18-RH']
MediumRouteSet_v1_id 3
beta ['G2-LH', 'G2-RH', 'F5-LH', 'F5-RH', 'B8-LH', 'F10-RH', 'E13-LH', 'H14-RH', 'E18-LH']
MediumRouteSet_v1_id 4
beta ['A5-LH', 'A5-RH', 'D5-RH', 'B8-LH', 'F10-RH', 'F12-LH', 'J13-RH', 'G15-LH', 'K18-RH']
MediumRouteSet_v1_id 5
beta ['A5-LH', 'C5-RH', 'B6-LH', 'E9-RH', 'D11-LH', 'I14-RH', 'E18-LH']
MediumRouteSet_v1_id 6
beta ['K5-LH', 'K5-RH', 'G6-LH', 'J9-RH', 'F14-LH', 'E18-RH']
MediumRouteSet_v1_id 7
beta ['E6-LH', 'E6-RH', 'F10-RH', 'C13-LH', 'D18-RH']
MediumRouteSet_v1_id 8
beta ['E6-LH', 'E6-RH', 'F10-RH', 'C13-LH', 'E16-RH', 'G18-LH']
MediumRouteSet_v1_id 9
beta ['E6-LH', 'E6-RH', 'H8-RH', 'C10-LH', 'G14-RH', 'D18-LH']
MediumRouteSet_v1_id 10
beta ['A5-LH', 'A5-RH', 'C5-LH', 'E8-RH', 'F10-LH', 'H12-RH', 'G15-LH', 'K18-RH']
MediumRouteSet_v1_id 11
beta ['C5-LH', 'C5-

**Space to play with**: 

* Initial conditions like x,a,c can be changed
* Should compare the similarity with the training set. 
* Package the DeepRouteSet with gradeNet to predict the grade

**References**

This program is adapted from Andrew Ng coursera's course Jazz music
The ideas presented in this notebook came primarily from three computational music papers cited below. The implementation here also took significant inspiration and used many components from Ji-Sung Kim's GitHub repository.

- Ji-Sung Kim, 2016, [deepjazz](https://github.com/jisungk/deepjazz)
- Jon Gillick, Kevin Tang and Robert Keller, 2009. [Learning Jazz Grammars](http://ai.stanford.edu/~kdtang/papers/smc09-jazzgrammar.pdf)
- Robert Keller and David Morrison, 2007, [A Grammatical Approach to Automatic Improvisation](http://smc07.uoa.gr/SMC07%20Proceedings/SMC07%20Paper%2055.pdf)
- François Pachet, 1999, [Surprising Harmonies](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.5.7473&rep=rep1&type=pdf)